<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/Federated_Learning%20/Notebooks/FL_Default_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#Importa Pytorch, Flower 
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
#Importa as bibliotecas necessárias
from collections import OrderedDict
from typing import Dict, Optional, List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
from torch.optim.adam import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [47]:
#Define o dispositivo de treinamento
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__} and Flower{fl.__version__}")

Training on cpu using Pytorch2.0.1+cu118 and Flower1.4.0


In [48]:
# Define os valores dos hiperparâmetros de treinamento
BATCH_SIZE =1024
NUM_CLIENTS= 2
optim_lr=0.001
local_epochs=3
num_rounds=2

In [49]:
#Carrega os dados
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    # Realiza a partição do dataset CIFAR10 em n clientes
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10 # 10% do conjunto para testes
        len_train = len(ds) - len_val
        lengths =[len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,num_workers=0))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False,num_workers=0))
        testloader = DataLoader(testset,batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
        num_examples = {"ds_train": len(ds_train)*BATCH_SIZE, "ds_val": len(ds_val)*BATCH_SIZE}

    return trainloaders, valloaders, testloader,num_examples
    
trainloaders, valloaders, testloader,num_examples = load_data()

Files already downloaded and verified
Files already downloaded and verified


In [50]:
# Define o Modelo de Rede Neural
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [51]:
# Define as funções de treino e teste
def train(net, trainloader, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(),lr=optim_lr)
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [52]:
# Define as funções de atualização de parâmetros do modelo de rede neural
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [53]:
# Define a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, num_example)-> None:
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.num_example = num_example

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader = trainloaders[int(cid)]
        valloader = valloaders[int(cid)]
        num_example = num_examples

        return FlowerClient(cid, net, trainloader, valloader, num_example)

In [54]:
# Define a função de agregação do aorendizado federado
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [55]:
# Inicializa o treino federado
# Cria a função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
# Define o dispositivo a ser utilizado pelos clientes
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)
     

INFO flwr 2023-06-11 23:15:27,107 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)


(launch_and_evaluate pid=7597) [Client 3] evaluate, config: {} [repeated 4x across cluster]


2023-06-11 23:15:36,459	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-11 23:15:38,996 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3897502924.0, 'memory': 7795005851.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3897502924.0, 'memory': 7795005851.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO flwr 2023-06-11 23:15:39,008 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-11 23:15:39,028 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=13554) 2023-06-11 23:15:46.646807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO flwr 2023-06-11 23:15:51,966 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one r

(launch_and_get_parameters pid=13554) [Client 0] get_parameters
(launch_and_fit pid=13554) [Client 1] fit, config: {}


(pid=13553) 2023-06-11 23:15:58.664841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

(launch_and_fit pid=13554) [Client 1] fit, config: {}
(launch_and_evaluate pid=13554) [Client 1] fit, config: {}


DEBUG flwr 2023-06-11 23:18:34,749 | server.py:232 | fit_round 2 received 2 results and 0 failures
DEBUG:flwr:fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-06-11 23:18:34,764 | server.py:168 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 2: strategy sampled 2 clients (out of 2)


(launch_and_evaluate pid=13554) [Client 1] evaluate, config: {}
(launch_and_fit pid=13553) [Client 1] evaluate, config: {}


DEBUG flwr 2023-06-11 23:18:41,475 | server.py:182 | evaluate_round 2 received 2 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 2 results and 0 failures
INFO flwr 2023-06-11 23:18:41,483 | server.py:147 | FL finished in 169.5081554999997
INFO:flwr:FL finished in 169.5081554999997
INFO flwr 2023-06-11 23:18:41,485 | app.py:218 | app_fit: losses_distributed [(1, 0.002078236937522888), (2, 0.0019356756925582887)]
INFO:flwr:app_fit: losses_distributed [(1, 0.002078236937522888), (2, 0.0019356756925582887)]
INFO flwr 2023-06-11 23:18:41,487 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-11 23:18:41,492 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.373), (2, 0.4136)]}
INFO:flwr:app_fit: metrics_distributed {'accuracy': [(1, 0.373), (2, 0.4136)]}
INFO flwr 2023-06-11 23:18:41,498 | app.py:221 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-06-11 23:18:41,

History (loss, distributed):
	round 1: 0.002078236937522888
	round 2: 0.0019356756925582887
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.373), (2, 0.4136)]}